In [7]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel
from kobert_tokenizer import KoBERTTokenizer
from torch.utils.data import Dataset, DataLoader

# 1. 지역 전처리 함수 정의
def process_location(location):
    major_regions = {
        '서울': ['서울', '중구', '여의도', '영등포', '강남', '서초', '종로', '마포', '강북', '강동', '송파', '은평', '동대문', '서대문', '성북', '용산'],
        '대전': ['대전', '유성', '서구', '동구', '중구'],
        '부산': ['부산', '해운대', '수영', '남구', '중구', '동구', '북구', '사하', '사상', '연제', '기장'],
        '대구': ['대구', '중구', '동구', '서구', '남구', '북구', '수성', '달서', '달성'],
        '충청도': ['충북', '충청북도', '충남', '충청남도', '청주', '천안', '공주', '논산', '제천', '아산', '서산'],
        '경기도': ['경기', '수원', '용인', '성남', '고양', '안산', '부천', '화성', '남양주', '의정부', '평택', '광명', '군포', '하남', '파주'],
        '경상도': ['경북', '경상북도', '경남', '경상남도', '창원', '김해', '양산', '거제', '진주', '포항', '경주', '울산'],
        '전라도': ['전북', '전라북도', '전남', '전라남도', '전주', '익산', '군산', '여수', '목포', '순천', '광주'],
        '강원도': ['강원', '강릉', '춘천', '원주', '속초'],
        '제주도': ['제주', '서귀포']
    }
    
    regions = [region.strip() for region in str(location).split(',') if region.strip()]
    result = []
    for region in regions:
        for major, subregions in major_regions.items():
            if any(subregion in region for subregion in subregions):
                result.append(region if region in subregions else major)
                break
        else:
            result.append('기타')
    
    return list(set(result))  # 중복 제거

In [10]:
# 2. 감성 예측 모델 정의 및 초기화
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bert_model = BertModel.from_pretrained('skt/kobert-base-v1')

class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=3, dr_rate=0.5):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.classifier = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(p=dr_rate)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        out = self.dropout(pooled_output)
        return self.classifier(out)

# 감성 예측 모델 초기화
model = BERTClassifier(bert_model, num_classes=3)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# 감성 예측 함수 정의
def predict_sentiment(text):
    inputs = tokenizer(text, max_length=64, padding='max_length', truncation=True, return_tensors="pt")
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probs = F.softmax(outputs, dim=1)
        pred_label = torch.argmax(probs, dim=1).cpu().numpy()[0]
    
    sentiment_map = {0: '부정', 1: '중립', 2: '긍정'}
    return sentiment_map[pred_label]

# 3. CSV 파일을 로드하고 각 기사에 대해 분석 수행
df = pd.read_csv('./NewsResult.csv')

# 결측값을 빈 문자열로 대체하여 문자열 결합 가능하게 처리
df['본문'] = df['본문'].fillna("").astype(str)
df['키워드'] = df['키워드'].fillna("").astype(str)

# 결과 저장을 위한 리스트 생성
processed_locations = []
sentiments = []

for idx, row in df.iterrows():
    location = row['위치']
    text = row['본문'] + " " + row['키워드']

    # 위치 전처리 및 감성 예측
    processed_locations.append(process_location(location))
    sentiments.append(predict_sentiment(text))

# 전처리 및 예측 결과를 데이터프레임에 추가
df['전처리된 위치'] = processed_locations
df['감성'] = sentiments

# 결과 확인
print(df[['본문', '위치', '전처리된 위치', '감성']].head())

# 필요 시, 결과 저장
df.to_csv('Processed_NewsResult.csv', index=False)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


                                                  본문       위치 전처리된 위치  감성
0  ■국세청 ◇서기관 승진 △국세청 혁신정책담당관실 이우진 △ 기획재정담당관실 박찬웅 ...  고광덕,김덕은    [기타]  긍정
1                                             [사진기사]        中    [기타]  중립
2  ◇산업통상자원부 \n \n△과장급 전보 \n \n▷정보관리담당관 김열규 ▷무역정책과...      NaN    [기타]  긍정
3  [이투데이] 세종=조아라 기자 (abc@etoday.co.kr)\n\n◇과장급 전보...       세종    [기타]  긍정
4  ◇국세청 △서기관 승진 ▷혁신정책담당관실 이우진 ▷기획재정담당관실 박찬웅 ▷감찰담당...      NaN    [기타]  긍정
